In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords, cmudict
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
import xgboost as xgb
nltk.download('cmudict')
nltk.download('punkt')
print('****Import Complete****')

****Import Complete****


[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\hlani\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hlani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Merging Data Sets
human_data = pd.read_csv('../Data/train_data.csv')
gpt3_data = pd.read_csv('../Data/essays_gpt3.5.csv') 
gpt4_data = pd.read_csv('../Data/essays_gpt4.csv')
df = pd.concat([human_data,gpt3_data,gpt4_data])
df = df.sample(frac=1).reset_index(drop=True)

df_test = pd.read_csv('../Challenge/test_essays.csv')
df_test = df_test.sample(frac=1).reset_index(drop=True)


# Readability Functions

In [3]:

# Syllable count in a word
def syllable_count(word, d):
    if word.lower() in d:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    else:
        # Fallback method for words not in cmudict
        return len(re.findall(r'[aeiouy]+', word.lower()))
# Initialize CMU Pronouncing Dictionary
d = cmudict.dict()

# Helper function to preprocess text and count syllables, words, and sentences
def preprocess_and_count(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    syllables = sum(syllable_count(word, d) for word in words)
    num_sentences = len(sentences)
    num_words = len([word for word in words if word.isalpha()])
    num_syllables = syllables
    num_chars = sum(len(word) for word in words if word.isalpha())
    return num_sentences, num_words, num_syllables, num_chars


# Flesch Reading Ease
def flesch_reading_ease(text):
    num_sentences, num_words, num_syllables, _ = preprocess_and_count(text)
    return 206.835 - 1.015 * (num_words / num_sentences) - 84.6 * (num_syllables / num_words)

# Flesch-Kincaid Grade Level
def flesch_kincaid_grade(text):
    num_sentences, num_words, num_syllables, _ = preprocess_and_count(text)
    return 0.39 * (num_words / num_sentences) + 11.8 * (num_syllables / num_words) - 15.59

# Gunning Fog Index
def gunning_fog(text):
    num_sentences, num_words, num_syllables, _ = preprocess_and_count(text)
    complex_words = sum(syllable_count(word, d) >= 3 for word in word_tokenize(text))
    return 0.4 * ((num_words / num_sentences) + 100 * (complex_words / num_words))

# SMOG Index
def smog_index(text):
    num_sentences, _, num_syllables, _ = preprocess_and_count(text)
    return 1.043 * (30 * (num_syllables / num_sentences))**0.5 + 3.1291

# Automated Readability Index
def automated_readability_index(text):
    num_sentences, num_words, _, num_chars = preprocess_and_count(text)
    return 4.71 * (num_chars / num_words) + 0.5 * (num_words / num_sentences) - 21.43

# Coleman-Liau Index
def coleman_liau_index(text):
    num_sentences, num_words, _, num_chars = preprocess_and_count(text)
    L = (num_chars / num_words) * 100
    S = (num_sentences / num_words) * 100
    return 0.0588 * L - 0.296 * S - 15.8



# Sentence Length Variability Feature

In [4]:
# Parse each essay into a list of words
df['parse'] = df['text'].str.split()
# Score each essay on Setence Length Variability
df['SLV'] = df['parse'].apply(lambda x: np.std([len(word_tokenize(sentence)) for sentence in x]))
# Remove Parse Data
df.drop(['parse'], axis=1, inplace=True)
df.SLV

# Parse each essay into a list of words
df_test['parse'] = df_test['text'].str.split()
# Score each essay on Setence Length Variability
df_test['SLV'] = df_test['parse'].apply(lambda x: np.std([len(word_tokenize(sentence)) for sentence in x]))
# Remove Parse Data
df_test.drop(['parse'], axis=1, inplace=True)
df_test.SLV

0    0.471405
1    0.471405
2    0.471405
Name: SLV, dtype: float64

# Conjunction Count Feature

In [5]:
conjunctions = [
    "and", "but", "or", "so", "yet", "nor", "for", "after", "although",
    "as", "because", "before", "if", "once", "since", "that", "though",
    "till", "unless", "while", "where", "whether", "because of", "in order that",
    "even though", "as long as", "as soon as", "just as", "so that", "in case",
    "now that", "as if", "provided that", "whereas", "inasmuch as", "whenever",
    "until", "while", "after all", "as though", "lest", "regardless", "apart from",
    "given that", "if only", "in case that", "in spite of", "on the condition that",
    "only if", "supposing", "as far as", "in the event that", "not to mention",
    "rather than", "such that", "to the extent that", "although", "despite",
    "much as", "whether or not", "assuming that", "besides", "conversely",
    "except that", "in order to", "like", "provided", "save that", "that is to say",
    "to the end that", "wherever", "whiles", "by the time", "even if",
    "on condition that", "so long as", "apart from that", "even when", "if then",
    "in as much as", "in spite of the fact that", "in the same way that",
    "not only but also", "notwithstanding", "presuming that", "rather", "seeing that",
    "unless and until", "whereas as", "whether or no", "as against", "as well as",
    "in accordance with", "in addition to", "in relation to", "in the light of",
    "not to speak of", "regardless of the fact that", "so as to", "with regard to"]
# Create a new column for the number of conjunctions in each essay
df["num_conjunctions"] = np.zeros(len(df))

# Use a regular expression to find conjunctions in the text
conjunctions_regex = r'\b(?:' + '|'.join(conjunctions) + r')\b'

# Count occurrences of each conjunction in the 'text' column
df['num_conjunctions'] = df['text'].str.count(conjunctions_regex)

df.num_conjunctions

# Create a new column for the number of conjunctions in each essay
df_test["num_conjunctions"] = np.zeros(len(df_test))

# Use a regular expression to find conjunctions in the text
conjunctions_regex = r'\b(?:' + '|'.join(conjunctions) + r')\b'

# Count occurrences of each conjunction in the 'text' column
df_test['num_conjunctions'] = df_test['text'].str.count(conjunctions_regex)

df_test.num_conjunctions


0    0
1    0
2    0
Name: num_conjunctions, dtype: int64

# Readability Features

In [6]:
df['flesch_reading_ease'] = df['text'].apply(flesch_reading_ease)
df['flesch_kincaid_grade'] = df['text'].apply(flesch_kincaid_grade)
df['gunning_fog'] = df['text'].apply(gunning_fog)
df['smog_index'] = df['text'].apply(smog_index)
df['automated_readability_index'] = df['text'].apply(automated_readability_index)
df['coleman_liau_index'] = df['text'].apply(coleman_liau_index)

# Applying readability tests to each essay
df_test['flesch_reading_ease'] = df_test['text'].apply(flesch_reading_ease)
df_test['flesch_kincaid_grade'] = df_test['text'].apply(flesch_kincaid_grade)
df_test['gunning_fog'] = df_test['text'].apply(gunning_fog)
df_test['smog_index'] = df_test['text'].apply(smog_index)
df_test['automated_readability_index'] = df_test['text'].apply(automated_readability_index)
df_test['coleman_liau_index'] = df_test['text'].apply(coleman_liau_index)

# Scaling Numerical Data

In [7]:
#scale numerical data
scaler = MinMaxScaler()
columns_scale = df.drop(['text', 'generated','id'], axis=1).columns
scaled_data = scaler.fit_transform(df[columns_scale])
scaled_df = pd.DataFrame(scaled_data, columns=columns_scale)
df_scaled = pd.concat([df[['id','prompt_id','text']],scaled_df], axis=1)


#scale numerical data
scaler = MinMaxScaler()
columns_test_scale = df_test.drop(['text','id'], axis=1).columns
scaled_test_data = scaler.fit_transform(df_test[columns_test_scale])
scaled_test_df = pd.DataFrame(scaled_test_data, columns=columns_test_scale)
df_test_scaled = pd.concat([df_test[['id','prompt_id','text']],scaled_test_df], axis=1)

df_scaled

,id,prompt_id,text,prompt_id,SLV,num_conjunctions,flesch_reading_ease,flesch_kincaid_grade,gunning_fog,smog_index,automated_readability_index,coleman_liau_index
0,2fe0b1f5,1,"Dear Senator, I know you have a lot of things ...",1.0,0.469139,0.169492,0.692876,0.155011,0.179135,0.188216,0.135622,0.331413
1,dfeb1efd,0,Title: Advantages of Limiting Car Usage and Em...,0.0,0.671944,0.211864,0.266352,0.357819,0.383491,0.376252,0.298950,0.753385
2,1a205da4,1,The electoral college is an unfair and outdate...,1.0,0.374249,0.279661,0.661915,0.180814,0.222008,0.225634,0.172200,0.399301
3,0a62093d,0,Within countries around the world people are c...,0.0,0.240413,0.593220,0.563614,0.305009,0.298885,0.404358,0.302786,0.425810
4,6e88a706,0,The effects of greenhouse gas emission began t...,0.0,0.404136,0.254237,0.732890,0.163175,0.171871,0.226453,0.171048,0.343539
...,...,...,...,...,...,...,...,...,...,...,...,...
2773,7959baab,1,"[Your Name]\r\n[Your Address]\r\n[City, State,...",1.0,0.476854,0.288136,0.494429,0.252524,0.270329,0.282595,0.227593,0.573268
2774,d5519d77,1,"Dear State Senator, I think that our president...",1.0,0.317192,0.347458,0.768200,0.173296,0.175751,0.258640,0.183173,0.262361
2775,d179fee2,1,"[Your Name]\r\n[Your Address]\r\n[City, State,...",1.0,0.485793,0.279661,0.359397,0.312761,0.331545,0.333442,0.274489,0.708897
2776,2f6621e0,1,"[Your Name]\r\n[Your Address]\r\n[City, State,...",1.0,0.487318,0.288136,0.390975,0.307183,0.313700,0.339882,0.283334,0.707005


# Creating Text Matrix 

In [8]:


tfidf_vectorizer = TfidfVectorizer(max_features=5000)
x_tfidf = tfidf_vectorizer.fit_transform(df['text'])
# Combine with Aditional Features
combined_x = scipy.sparse.hstack([x_tfidf, df.drop(['id','prompt_id','text'], axis=1).values])

x_test_tfidf = tfidf_vectorizer.transform(df_test['text'])
# Combine with Aditional Features
combined_test_x = scipy.sparse.hstack([x_test_tfidf, df_test.drop(['id','text'], axis=1).values])
print(x_test_tfidf)

# Polish final secitons for model


In [9]:
X_train = combined_x
X_test = combined_test_x
y_train = df['generated']


# Fine Tuning Hyper Parameters

In [10]:
# Parameter Grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}
xgb_classifier = xgb.XGBClassifier(eval_metric='logloss',tree_method='gpu_hist', gpu_id=0)

grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=3, n_jobs=-1,verbose=2)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)


Fitting 3 folds for each of 972 candidates, totalling 2916 fits


C:\Users\hlani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [06:04:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
C:\Users\hlani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [06:04:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8}


# Build Model and Test

In [11]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [12]:

probabilities = xgb_model.predict_proba(X_test)[:, 1]  # For probabilities
predictions = xgb_model.predict(X_test)  # For class labels

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': df_test['id'],  # Ensure this is the correct ID column from your test set
    'generated': probabilities
})

# Save to CSV file
submission_filename = 'submission.csv'
submission_df.to_csv(submission_filename, index=False)

submission_df

,id,generated
0,1111bbbb,0.000667
1,2222cccc,0.000667
2,0000aaaa,0.999347
